In [1]:
import json

In [2]:
with open("../../gis/google/Takeout/ロケーション履歴/ロケーション履歴.json") as f:
    history = json.load(f)

In [3]:
import datetime
import pandas as pd

locations = pd.DataFrame.from_dict(history["locations"])
locations["datetime"] = locations.timestampMs.map(lambda x: datetime.datetime.fromtimestamp(int(x)/1000))

import pyproj

epsg_from, epsg_to = 4326, 6677
tr = pyproj.Transformer.from_proj(epsg_from, epsg_to)
xys = [tr.transform(lat/10**7, lng/10**7) for lat, lng in zip(locations.latitudeE7, locations.longitudeE7)]    
locations[["x", "y"]] = xys
locations

,timestampMs,latitudeE7,longitudeE7,accuracy,activity,velocity,altitude,verticalAccuracy,heading,datetime,x,y
0,1373458943759,357028875,1395619739,2868,NaN,NaN,NaN,NaN,NaN,2013-07-10 21:22:23.759,-32929.257007,-24555.785773
1,1373459484329,357010784,1394978473,2614,NaN,NaN,NaN,NaN,NaN,2013-07-10 21:31:24.329,-33112.029868,-30359.418153
2,1373459665257,357005741,1394816046,2820,NaN,NaN,NaN,NaN,NaN,2013-07-10 21:34:25.257,-33162.834373,-31829.488621
3,1373459845824,357003463,1394763803,2753,NaN,NaN,NaN,NaN,NaN,2013-07-10 21:37:25.824,-33186.400987,-32302.352467
4,1373459966250,357005075,1394638697,2609,NaN,NaN,NaN,NaN,NaN,2013-07-10 21:39:26.250,-33164.328855,-33434.432947
...,...,...,...,...,...,...,...,...,...,...,...,...
383245,1607044343720,355853153,1397253045,87,NaN,NaN,41.0,2.0,NaN,2020-12-04 10:12:23.720,-46001.436758,-9790.038276
383246,1607044464083,355853153,1397253045,1653,"[{'timestampMs': '1607044346522', 'activity': ...",NaN,41.0,2.0,NaN,2020-12-04 10:14:24.083,-46001.436758,-9790.038276
383247,1607044585132,356141415,1397360164,1083,"[{'timestampMs': '1607044588597', 'activity': ...",NaN,NaN,NaN,NaN,2020-12-04 10:16:25.132,-42804.454420,-8816.117473
383248,1607044709718,356274644,1397390657,110,NaN,NaN,NaN,NaN,NaN,2020-12-04 10:18:29.718,-41326.668308,-8538.459440


In [4]:
import numpy as np

In [5]:
def load_data(df, n_prev=100):
    xs = df.x.values
    ys = df.y.values
    ts = df.timestampMs.map(int).values / 1000
    diffs = ts[1:] - ts[:-1]

    n_rows = len(df)    
    data = np.zeros((n_rows, 3))
    data[:, 0] = xs
    data[:, 1] = ys
    data[:-1, 2] = diffs
    data[-1, 2] = None

    n_ret = n_rows - n_prev
    X = np.zeros((n_ret, n_prev, 3))
    for i in range(n_ret):
        X[i, :, :] = data[i:i+n_prev, :]
    y = data[n_prev:, :2]
    return X, y

In [6]:
X, y = load_data(locations)

In [7]:
np.all(X[0,1:] == X[1,:-1]), \
np.all(X[1,-1,:2] == y[0]), \
np.all(y[-1] == locations[["x", "y"]].values[-1])

(True, True, True)

In [8]:
n_data = len(X)
n_train, n_val = int(n_data * 0.7), int(n_data * 0.15)
n_test = n_data - n_train - n_val

In [9]:
X_train, X_val, X_test = X[0:n_train], X[n_train:n_train+n_val], X[n_train+n_val:]
y_train, y_val, y_test = y[0:n_train], y[n_train:n_train+n_val], y[n_train+n_val:]

In [10]:
from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.layers.recurrent import LSTM
from keras.losses import mean_squared_error

In [11]:
n_lstm = 300

model = Sequential()
model.add(LSTM(n_lstm, input_shape=X.shape[1:]))
model.add(Dense(2))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 300)               364800    
_________________________________________________________________
dense (Dense)                (None, 2)                 602       
Total params: 365,402
Trainable params: 365,402
Non-trainable params: 0
_________________________________________________________________


In [12]:
model.compile(
    loss=mean_squared_error,
    optimizer="adam",
    metrics=["mean_squared_error"]
)
model.fit(
    X_train, y_train, validation_data=(X_val, y_val), batch_size=32, epochs=3
)

Epoch 1/3
8382/8382 [==============================] - 1453s 173ms/step - loss: 55862108160.0000 - mean_squared_error: 55862108160.0000 - val_loss: 1529157632.0000 - val_mean_squared_error: 1529157632.0000
Epoch 2/3
2310/8382 [=======>......................] - ETA: 16:30 - loss: 63287549952.0000 - mean_squared_error: 63287549952.0000

KeyboardInterrupt: 